In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "HOnK5355mmx"
   },
   "outputs": [],
   "source": [
    "from pathlib import Path\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "from sklearn.model_selection import train_test_split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "p0AkGBQX5_Ei"
   },
   "outputs": [],
   "source": [
    "data_dir = Path(\"data\")\n",
    "assert data_dir.exists(), \"Expected /data to be present\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "u3MHy9aOuC9-"
   },
   "outputs": [],
   "source": [
    "# Load the full training dataset\n",
    "df_full = pd.read_csv(data_dir / \"train.csv\")\n",
    "\n",
    "# Split into 80% train and 20% validation (stratified by target)\n",
    "df_train, df_val = train_test_split(df_full, test_size=0.2, random_state=42, stratify=df_full['Churn'])\n",
    "\n",
    "print(f\"Train set size: {len(df_train)}\")\n",
    "print(f\"Validation set size: {len(df_val)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "BaxH-Ibhr2Ab"
   },
   "outputs": [],
   "source": [
    "class ChurnPredictor:\n",
    "    def __init__(self):\n",
    "        pass\n",
    "\n",
    "    def fit(self, train_df):\n",
    "        \"\"\"\n",
    "        Takes the full train_df (including 'Churn' column) and trains the model.\n",
    "        You should perform any necessary preprocessing, encoding, scaling, imputation, etc.\n",
    "        \"\"\"\n",
    "        pass\n",
    "\n",
    "    def predict(self, df):\n",
    "        \"\"\"\n",
    "        Takes a dataframe (without the 'Churn' column) and returns binary predictions.\n",
    "        Returns: NumPy array or list of 0 (stay) or 1 (churn)\n",
    "        \"\"\"\n",
    "        # Placeholder - replace with real prediction logic\n",
    "        predictions = [0 for _ in range(len(df))]\n",
    "        return np.array(predictions)\n",
    "\n",
    "    def predict_proba(self, df):\n",
    "        \"\"\"\n",
    "        Takes a dataframe (without the 'Churn' column) and returns probabilities.\n",
    "        Returns: NumPy array of shape (n_samples, 2) where column 1 is P(churn)\n",
    "        \"\"\"\n",
    "        # Placeholder - replace with real probability logic\n",
    "        n = len(df)\n",
    "        return np.column_stack([np.full(n, 0.5), np.full(n, 0.5)])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "86qmcEySvpAB"
   },
   "outputs": [],
   "source": [
    "# --- EXECUTION FLOW ---\n",
    "\n",
    "# Initialize the class\n",
    "predictor = ChurnPredictor()\n",
    "\n",
    "# Train on the training split\n",
    "predictor.fit(df_train)\n",
    "\n",
    "# Evaluate on validation split\n",
    "y_true_val = (df_val['Churn'] == 'Yes').astype(int)\n",
    "y_pred_proba_val = predictor.predict_proba(df_val.drop('Churn', axis=1))[:, 1]\n",
    "\n",
    "from sklearn.metrics import roc_auc_score\n",
    "val_auc = roc_auc_score(y_true_val, y_pred_proba_val)\n",
    "print(f\"Validation Set ROC AUC: {val_auc:.4f}\")\n",
    "\n",
    "# Get predictions for the validation set\n",
    "final_predictions = predictor.predict(df_val.drop('Churn', axis=1))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "lVOcrF-xvad7"
   },
   "outputs": [],
   "source": [
    "!mkdir -p /results\n",
    "\n",
    "# Write the class definition to utils.py\n",
    "with open('/results/utils.py', 'w') as f:\n",
    "    f.write('''\n",
    "class ChurnPredictor:\n",
    "    def __init__(self):\n",
    "        pass\n",
    "\n",
    "    def fit(self, train_df):\n",
    "        pass\n",
    "\n",
    "    def predict(self, df):\n",
    "        pass\n",
    "\n",
    "    def predict_proba(self, df):\n",
    "        pass\n",
    "''')\n",
    "\n",
    "print(\"Wrote ChurnPredictor definition to /results/utils.py\")"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "name": "python3"
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}